<a href="https://colab.research.google.com/github/heejvely/second-projecct/blob/main/compare_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [ ]:
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os

In [ ]:
# colab 세션에 파일을 업로드해도 되고, google-drive mount 후 해당 파일들이 있는 폴더로 옮긴 후 진행하셔도 됩니다.
df = pd.read_csv('final_train.txt', sep='\t', names=['label', 'comment'])
df_val = pd.read_csv('final_validation.txt', sep='\t', names=['label', 'comment'])
df_test = pd.read_csv('final_test.txt', sep='\t', names=['label', 'comment'])

In [ ]:
df.head()

,label,comment
0,0,동물보다 동물 모형이 더 많아요!
1,0,아....방학이라 사람이 넘 많아요. 대기시간 180분. 헐
2,0,공원도 뭣도 아닌 곳사람이 지나다니는 길에 콘크리트 화분이 수백개미치지 않고서야어찌...
3,0,생각보다 다양하거나 크진않았던거같아요ㅜ
4,0,남대문 시장에 가는것은 순전히 갈치조림 먹는거랑 지나가는길에 가는 정도!! 사실 크...


In [ ]:
comments = df.comment

In [ ]:
len(df)

9600

In [ ]:
# train 데이터를 기준으로 bpe 모델을 훈련해 줍니다.
# 기타 토크나이저를 사용해도 무방합니다.
vocab_filename = "travel_encoder"

if not os.path.isfile(vocab_filename+".subwords"):
    encoder = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
        comments, target_vocab_size = 20000)

    encoder.save_to_file(vocab_filename)
else:
    encoder = tfds.deprecated.text.SubwordTextEncoder.load_from_file(vocab_filename)


In [ ]:
encoder.encode(comments[0])

[7567, 27, 1665, 14212, 16197, 20, 146, 16198]

In [ ]:
comments[0]

'동물보다 동물 모형이 더 많아요!'

In [ ]:
# encoder로 commnet를 encode해준 뒤, 각 df별로 "comment_encoded"라는 column에 추가해 줍니다.
df["comment_encoded"] = df.comment.apply(encoder.encode)
df_val["comment_encoded"] = df_val.comment.apply(encoder.encode)
df_test["comment_encoded"] = df_test.comment.apply(encoder.encode)

In [ ]:
# encoder에 대한 정보입니다.
vocab_size = encoder.vocab_size
print(vocab_size)
encoder.subwords[:30]

16421


['. ',
 '너무_',
 ', ',
 '도_',
 '는_',
 '이_',
 '..',
 '고_',
 '수_',
 '을_',
 '에_',
 '많이_',
 '은_',
 '가_',
 '...',
 '.. ',
 '좀_',
 '사람이_',
 '의_',
 '더_',
 '서_',
 '좋은_',
 '좋아요',
 '있는_',
 '잘_',
 '를_',
 '다_',
 '만_',
 '게_',
 '한_']

In [ ]:
df.head()

,label,comment,comment_encoded
0,0,동물보다 동물 모형이 더 많아요!,"[7567, 27, 1665, 14212, 16197, 20, 146, 16198]"
1,0,아....방학이라 사람이 넘 많아요. 대기시간 180분. 헐,"[155, 151, 4135, 18, 82, 146, 1, 3211, 16149, ..."
2,0,공원도 뭣도 아닌 곳사람이 지나다니는 길에 콘크리트 화분이 수백개미치지 않고서야어찌...,"[15619, 16400, 16338, 16328, 4, 660, 43, 18, 1..."
3,0,생각보다 다양하거나 크진않았던거같아요ㅜ,"[36, 6079, 241, 5330, 9142, 2343, 517]"
4,0,남대문 시장에 가는것은 순전히 갈치조림 먹는거랑 지나가는길에 가는 정도!! 사실 크...,"[6123, 2487, 7992, 13, 1880, 255, 276, 5167, 1..."


In [ ]:
# train, test별로 input과 target을 분리해 줍니다.
# 0번 column: label, 2번 column: encoded comment
x_train, y_train = df.iloc[:,2], df.iloc[:,0]
x_val, y_val = df_val.iloc[:,2], df_val.iloc[:,0]
x_test, y_test = df_test.iloc[:,2], df_test.iloc[:,0]

In [ ]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [ ]:
# one-hot vector로 치환
y_train_categorical = tf.keras.utils.to_categorical(y_train)
y_val_categorical = tf.keras.utils.to_categorical(y_val)
y_test_categorical = tf.keras.utils.to_categorical(y_test)
y_train_categorical

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
# 훈련에 들어갈 수 있게 데이터를 변환해 줍니다.

from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 200
x_train = pad_sequences(x_train, padding="post", maxlen=maxlen)
x_val = pad_sequences(x_val, padding="post", maxlen=maxlen)
x_test = pad_sequences(x_test, padding="post", maxlen=maxlen)

In [ ]:
x_train.shape

(9600, 200)

In [ ]:
x_val.shape

(1200, 200)

In [ ]:
x_test.shape

(1200, 200)

In [ ]:
# test class 비중 파악
print(sum(y_test == 0)) # 나쁨
print(sum(y_test == 1)) # 좋음
print(sum(y_test == 2)) # 보통

400
400
400


In [ ]:
sum(y_test == 1) / len(y_test) # '좋음'만 할 경우의 정확도

0.3333333333333333

## DNN 기반의 classifier

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=maxlen))
model.add(GlobalAveragePooling1D())
model.add(Dense(3))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x=x_train, y=y_train_categorical, validation_data=(x_val, y_val_categorical), epochs=10)

Epoch 1/10
300/300 [==============================] - 3s 3ms/step - loss: 1.0984 - accuracy: 0.3514 - val_loss: 1.0972 - val_accuracy: 0.3525
Epoch 2/10
300/300 [==============================] - 1s 3ms/step - loss: 1.0962 - accuracy: 0.3705 - val_loss: 1.0955 - val_accuracy: 0.3508
Epoch 3/10
300/300 [==============================] - 1s 3ms/step - loss: 1.0941 - accuracy: 0.3656 - val_loss: 1.0937 - val_accuracy: 0.4100
Epoch 4/10
300/300 [==============================] - 1s 3ms/step - loss: 1.0918 - accuracy: 0.4334 - val_loss: 1.0915 - val_accuracy: 0.4183
Epoch 5/10
300/300 [==============================] - 1s 3ms/step - loss: 1.0890 - accuracy: 0.4768 - val_loss: 1.0890 - val_accuracy: 0.4067
Epoch 6/10
300/300 [==============================] - 1s 3ms/step - loss: 1.0858 - accuracy: 0.4999 - val_loss: 1.0860 - val_accuracy: 0.4242
Epoch 7/10
300/300 [==============================] - 1s 3ms/step - loss: 1.0820 - accuracy: 0.4564 - val_loss: 1.0826 - val_accuracy: 0.4917
Epoch 

In [ ]:
model.evaluate(x_test, y_test_categorical)

38/38 [==============================] - 0s 2ms/step - loss: 1.0686 - accuracy: 0.5308


[1.0686291456222534, 0.5308333039283752]

## Conv1D 기반의 classifier(TextCNN)

In [ ]:
model2 = tf.keras.Sequential()
model2.add(Embedding(vocab_size, 128, input_length=maxlen))
model2.add(tf.keras.layers.Conv1D(32, 7, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D(5))
model2.add(tf.keras.layers.Conv1D(32, 7, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D())
model2.add(tf.keras.layers.Conv1D(32, 7, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D())
model2.add(GlobalAveragePooling1D())
model2.add(Dense(3))

model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history2 = model2.fit(x=x_train, y=y_train_categorical, validation_data=(x_val, y_val_categorical), epochs=10)

Epoch 1/10
300/300 [==============================] - 16s 5ms/step - loss: 1.0952 - accuracy: 0.3706 - val_loss: 1.0881 - val_accuracy: 0.3692
Epoch 2/10
300/300 [==============================] - 1s 4ms/step - loss: 1.0768 - accuracy: 0.4231 - val_loss: 1.0664 - val_accuracy: 0.4308
Epoch 3/10
300/300 [==============================] - 1s 4ms/step - loss: 1.0432 - accuracy: 0.4480 - val_loss: 1.0268 - val_accuracy: 0.4667
Epoch 4/10
300/300 [==============================] - 1s 4ms/step - loss: 0.9451 - accuracy: 0.5241 - val_loss: 0.9288 - val_accuracy: 0.5517
Epoch 5/10
300/300 [==============================] - 1s 4ms/step - loss: 0.7863 - accuracy: 0.6154 - val_loss: 0.8546 - val_accuracy: 0.5875
Epoch 6/10
300/300 [==============================] - 1s 4ms/step - loss: 0.6581 - accuracy: 0.6844 - val_loss: 0.8544 - val_accuracy: 0.5967
Epoch 7/10
300/300 [==============================] - 1s 4ms/step - loss: 0.5713 - accuracy: 0.7320 - val_loss: 0.8707 - val_accuracy: 0.6142
Epoch

In [ ]:
model2.evaluate(x_test, y_test_categorical)

38/38 [==============================] - 0s 2ms/step - loss: 1.1994 - accuracy: 0.5925


[1.1993823051452637, 0.5924999713897705]

## LSTM 기반의 classifier

In [ ]:
model3 = tf.keras.Sequential()
model3.add(Embedding(vocab_size, 64, input_length=maxlen))
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, activation='relu')))
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, activation='relu')))
model3.add(tf.keras.layers.Dense(8, activation='relu'))
model3.add(tf.keras.layers.Dense(3))

model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history3 = model3.fit(x=x_train, y=y_train_categorical, validation_data=(x_val, y_val_categorical), epochs=4)

Epoch 1/4
300/300 [==============================] - 164s 538ms/step - loss: 1.0970 - accuracy: 0.3441 - val_loss: 1.0928 - val_accuracy: 0.4008
Epoch 2/4
300/300 [==============================] - 160s 535ms/step - loss: 1.0821 - accuracy: 0.4080 - val_loss: 1.0715 - val_accuracy: 0.4450
Epoch 3/4
300/300 [==============================] - 159s 529ms/step - loss: 1.0654 - accuracy: 0.4652 - val_loss: 1.0571 - val_accuracy: 0.4725
Epoch 4/4
300/300 [==============================] - 158s 526ms/step - loss: 1.0349 - accuracy: 0.4943 - val_loss: 1.0384 - val_accuracy: 0.5083


In [ ]:
model3.evaluate(x_test, y_test_categorical)

38/38 [==============================] - 3s 85ms/step - loss: 1.0449 - accuracy: 0.4900


[1.0449280738830566, 0.49000000953674316]

In [ ]:
model3.predict(x_test)